Generate separated files for each book containing the list of sentences. Sentence divisions is performed in a semi-manual way using specific patterns: only full stop is included when it is preceded by a token longer than 2 digits (i.e., p., M., but also ea). Inconsistencies and errors are present in the sentence split. The sentence count is at a book level: this means that sentence 23 in Book 4 is the 23rd sentence in the book.

In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
path = "data/intermediate/xml_Clerice_to_csv.csv"

In [3]:
xml_Clerice_to_csv = pd.read_csv(path, encoding='utf-8')

In [4]:
## extract the book from the reference and create a new column
book_column = []
pattern = r'(\d+)(?:\.(\d))?\.'

for i, reference in enumerate(xml_Clerice_to_csv['reference']):
    
    match = re.search(pattern,reference)
    book_column.append(int(match[1]))
    
xml_Clerice_to_csv['book'] = book_column

In [5]:
punctuation = ['.']

In [6]:
sentence_column = [] ## create a new column for the sentence count

In [7]:
for i in range(1,38): ## for each book of the NH
        
    # filter_book = LatLemm[LatLemm['book'] == i]
    filter_book = xml_Clerice_to_csv[xml_Clerice_to_csv['book'] == i]
    filter_book = filter_book.reset_index()
    
    count_sentence = 0
    
    for index,token in enumerate(filter_book['token']):
        sentence_column.append(count_sentence)
        
        if token in punctuation:
            
            if len(filter_book['token'][index-1]) < 3: ## if the full stop is preceded by a token shorter than 3 digits
                print(filter_book['index'][index-1], filter_book['token'][index-1]) ## do not increment the sentence count
                
            else : count_sentence = count_sentence + 1

837 T
1534 ea
43 9
64 13
108 22
143 28
319 65
374 75
435 87
61 21
86 .
87 .
93 .
94 .
100 .
101 .
106 .
107 .
116 .
117 .
126 .
127 .
128 .
42 14
76 25
84 28
125 .
126 .
132 .
133 .
138 .
139 .
148 .
149 .
158 .
159 .
27 5
61 13
73 18
79 20
111 )
136 .
137 .
143 .
144 .
150 .
151 .
164 .
165 .
174 .
175 .
243 o
27 3
127 30
302 o
273 T
120 23
362 70
62 12
394 et
154 32
285 52
287 53
349 65
364 67
441 om
443 E
448 om
148 28
402 ea
716 96
6 l
84 12
223 48
117 24
151 DX
179 32
193 36
206 39
128 30
404 74
126 29
155 35
61 15
111 33
234 69
281 87
70 15
81 19
165 58
199 61
129 I
152 II
157 I
244 72
279 II
296 II
86 )
222 VI
243 II
257 29
260 )
277 II
344 38
360 VI
384 II
423 42
611 54
703 29
792 VI
56 8
82 II
219 I
292 V
300 51
329 I
332 II
335 II
338 I
341 I
356 V
359 I
442 I
91 VI
94 II
99 I
102 I
115 22
216 46
223 II
226 I
229 I
232 II
241 I
246 II
301 57
335 70
350 II
372 V
375 I
119 II
122 II
125 II
144 VI
179 II
194 68
201 II
208 X
213 LI
258 II
261 II
264 VI
295 91
393 I
396 I
435 I
44

In [8]:
xml_Clerice_to_csv['sentence'] = sentence_column

In [9]:
xml_Clerice_to_csv.to_csv('data/intermediate/NH_lat_groupedsentences.csv', index=False)

In [10]:
## test
## filter all the rows containing sentence = 0
filter_book37 = xml_Clerice_to_csv[xml_Clerice_to_csv['book'] == 37]
print_sent_0 = filter_book37[filter_book37['sentence'] == 0]

## concatenate the tokens in filtered_rows
concatenate_string = ' '.join(print_sent_0['token'].astype(str))
concatenate_string

'Ut nihil institute operi desit , gemmae supersunt et in artum coacta rerum naturae maiestas , multis nulla parte mirabilior .'

In [11]:
## write files containing the sentences
for i in range(1,38): ## for each book of the NH
        
    filter_book = xml_Clerice_to_csv[xml_Clerice_to_csv['book'] == i]
    name='data/intermediate/books/latbook'+str(i)
    with open(name, "w+", encoding='utf-8') as file:
        for i in range(0, filter_book['sentence'].max()+1):
            filter_rows = filter_book[filter_book['sentence'] == i]
            concatenate_sentence = ' '.join(filter_rows['token'].astype(str))
            file.write(f"{concatenate_sentence}\n")

In [14]:
print('NOW YOU NEED TO RUN Use_SimAlign_Tokens_and_Project_annotations.ipynb OR vecalign.py?? ')

NOW YOU NEED TO RUN Use_SimAlign_Tokens_and_Project_annotations.ipynb.


In [12]:
#takes input generated in the Use_SimAlign_Tokens_and_Project_annotations.ipynb notebook. 
path = "data/intermediate/NH_lat_projected.csv"

In [13]:
NH_projected = pd.read_csv(path, encoding='utf-8', dtype={'reference': 'str'})

FileNotFoundError: [Errno 2] No such file or directory: 'data/intermediate/NH_lat_projected.csv'

In [ ]:
NH_projected

In [ ]:
index_in_sentence_column = []
index_in_sentence = 0

for i,token in enumerate(NH_projected['token']):
    print(i)
    if i == 0:
        index_in_sentence_column.append(index_in_sentence)
    if i > 0:
        if NH_projected['sentence'][i] == NH_projected['sentence'][i-1]: ## it is still in the same sentence
            index_in_sentence = index_in_sentence+1 ## increment
            index_in_sentence_column.append(index_in_sentence)
        else : 
            index_in_sentence = 0 # reset the index
            index_in_sentence_column.append(index_in_sentence)

In [ ]:
len(index_in_sentence_column)

In [ ]:
NH_projected['index_in_sentence'] = index_in_sentence_column

In [ ]:
NH_projected

In [ ]:
NH_projected.to_csv('NH_projected_index_sentences.csv',index=False)

In [ ]:
NH_projected_sentences = NH_projected[['index_in_sentence']].copy()

In [ ]:
NH_projected_sentences

In [ ]:
NH_projected_sentences.to_csv('index_in_sentences.csv',index=False)

In [ ]:
path = "/Users/u0154817/OneDrive - KU Leuven/Documents/KU Leuven/PhD project 'Greek Spaces in Roman Times'/Data_Extraction/Outputs/table_book_(chapter)_chapter_paragraph.csv"

In [ ]:
table_of_references = pd.read_csv(path, delimiter=';', encoding='utf-8', dtype={'reference': 'str'})

In [ ]:
table_of_references.dtypes

In [ ]:
table_of_references

In [ ]:
book_column = []
chapter_column = []
chapter_in_bracket_column = []

for book in range(1,38): ## for each book of the NH
        
    filter_book = table_of_references[table_of_references['Book'] == book]
    filter_book = filter_book.reset_index()
    
    unique_chapters = filter_book['Chapter'].unique().tolist() ## get a unique list of chapter numbers
    chapter_column.extend(unique_chapters)
    print(book, unique_chapters)
    
    for chapter in unique_chapters:
        book_column.append(book)
        temp_chapter_in_brackets = []
        
        for i,value in enumerate(filter_book['Chapter']):
            if value == chapter:
                if filter_book['(Chapter)'][i] in temp_chapter_in_brackets:
                    continue
                else : temp_chapter_in_brackets.append(filter_book['(Chapter)'][i])
        
        chapter_in_bracket_column.append(temp_chapter_in_brackets)       

In [ ]:
reference = pd.DataFrame({'book': book_column, 'chapter_Perseus': chapter_column, '(chapters)_ToposText': chapter_in_bracket_column})
reference.head(50)

In [ ]:
reference.to_csv('chapter_correspondences.csv',index=False)